# Overview

This notebook guides you through the setup and execution of the GOLD layer in your data pipeline using Snowflake and Iceberg tables. The GOLD layer focuses on curating aggregated and analytics-ready tables by leveraging the curated data from the SILVER and BRONZE layers.

You will:
- Import required Python packages and establish a Snowflake session.
- Set up user-specific variables, roles, databases, and schemas for the GOLD layer.
- Create dynamic Iceberg tables for key business entities and aggregations.
- Join and enrich data from the SILVER and BRONZE layers to produce final reporting tables.
- Create summary views for analytics and dashboarding.

Follow the instructions and code cells to complete the GOLD pipeline and prepare your data for business intelligence and advanced analytics.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


# Set User Number

Update the cell below with your unique user number for this lab. This ensures that all resources you create are isolated and do not conflict with those of other users.

**Usernum = '999'**

In [ ]:
usernum = str('<INSERT USER NUMBER>')

# Set User-Specific Variables

This section defines variables for your username, role, database, and schema. These variables will be used throughout the notebook to ensure all operations are performed in your dedicated environment.

In [ ]:


SET USERNAME = 'HOL_USER_' || {{usernum}};
SELECT $USERNAME;


In [ ]:
SET HOLROLE = $USERNAME || '_FULL_ROLE';
SET DB_NAME = $USERNAME || '_DB';
SET SCHEMANAME = 'GOLD';

In [ ]:
USE ROLE IDENTIFIER($HOLROLE);
USE DATABASE IDENTIFIER($DB_NAME);
USE SCHEMA IDENTIFIER($SCHEMANAME);

# Creating the SCHEDULE Table

This step creates a dynamic Iceberg table named `SCHEDULE` in the GOLD schema. The table is populated by selecting the latest schedule record for each unique schedule key from the SILVER layer, ensuring only the most relevant schedule information is retained for analytics.

In [ ]:
from snowflake.snowpark.functions import row_number, col,desc
from snowflake.snowpark.window import Window
# Read the table
df = session.table("silver.schedule_00")

# Define the window specification
window_spec = Window.partition_by("schedule_key").order_by("ORIGIN_LAT_LON")

# Add row number and filter
result_df = df.with_column("rn", row_number().over(window_spec)) \
              .filter(col("rn") == 1) \
              .drop("rn")

sch_iceberg_config = {
    "external_volume": "iceberg_hol_gold_vol",
    "catalog": "SNOWFLAKE",
    "base_location": f"hol_user_{usernum}/schedule/",
    "catalog_sync": "iceberg_hol_oc_int"
}

result_df.write.save_as_table("schedule",mode='overwrite',iceberg_config = sch_iceberg_config)

# Creating the TRAIN_ACTIVATIONS Table

This cell creates a dynamic Iceberg table named `TRAIN_ACTIVATIONS` by selecting the most recent activation event for each train from the SILVER layer. This ensures that only the latest activation record per train is available for downstream analytics.

In [ ]:
# Read the table
df = session.table("silver.train_activations_01")

# Define the window specification
window_spec = Window.partition_by("TRAIN_ID").order_by(desc("CREATION_TIMESTAMP"))

# Add row number and filter
ta_result_df = df.with_column("rn", row_number().over(window_spec)) \
              .filter(col("rn") == 1) \
              .drop("rn")

ta_iceberg_config = {
    "external_volume": "iceberg_hol_gold_vol",
    "catalog": "SNOWFLAKE",
    "base_location": f"hol_user_{usernum}/schedule/",
    "catalog_sync": "iceberg_hol_oc_int"
}

ta_result_df.write.save_as_table("TRAIN_ACTIVATIONS",mode='overwrite',iceberg_config = ta_iceberg_config)

# Creating the TRAIN_MOVEMENTS Table

This step creates a dynamic Iceberg table named `TRAIN_MOVEMENTS` by joining movement events from the SILVER layer with activation, schedule, and location data. The resulting table provides a comprehensive, enriched view of each train movement, including schedule, route, and geospatial information.

In [ ]:
CREATE OR REPLACE DYNAMIC ICEBERG TABLE TRAIN_MOVEMENTS
  TARGET_LAG = '1 minute'
  WAREHOUSE = hol_user_{{usernum}}_wh
  EXTERNAL_VOLUME= 'iceberg_hol_gold_vol'
  CATALOG = 'SNOWFLAKE'
  BASE_LOCATION = 'hol_user_{{usernum}}/train_movements/'
  CATALOG_SYNC = iceberg_hol_oc_int
AS
SELECT  TM.MSG_KEY AS MSG_KEY,
        TM.MSG_HEADER AS MSG_HEADER,
        TM.EVENT_TYPE AS EVENT_TYPE,
        TM.GBTT_TIMESTAMP AS GBTT_TIMESTAMP,
        TM.ORIGINAL_LOC_STANOX AS ORIGINAL_LOC_STANOX,
        TM.PLANNED_TIMESTAMP AS PLANNED_TIMESTAMP,
        TM.TIMETABLE_VARIATION AS TIMETABLE_VARIATION,
        TM.ORIGINAL_LOC_TIMESTAMP AS ORIGINAL_LOC_TIMESTAMP,
        TM.CURRENT_TRAIN_ID AS CURRENT_TRAIN_ID,
        TM.DELAY_MONITORING_POINT AS DELAY_MONITORING_POINT,
        TM.NEXT_REPORT_RUN_TIME AS NEXT_REPORT_RUN_TIME,
        TM.REPORTING_STANOX AS REPORTING_STANOX,
        TM.ACTUAL_TIMESTAMP AS ACTUAL_TIMESTAMP,
        TM.CORRECTION_IND AS CORRECTION_IND,
        TM.EVENT_SOURCE AS EVENT_SOURCE,
        TM.TRAIN_FILE_ADDRESS AS TRAIN_FILE_ADDRESS,
        TM.PLATFORM AS PLATFORM,
        TM.DIVISION_CODE AS DIVISION_CODE,
        TM.TRAIN_TERMINATED AS TRAIN_TERMINATED,
        TM.TRAIN_ID AS TRAIN_ID,
        TM.OFFROUTE_IND AS OFFROUTE_IND,
        TM.VARIATION AS VARIATION,
        TM.LATE_IND AS LATE_IND,
        TM.VARIATION_STATUS AS VARIATION_STATUS,
        TM.TRAIN_SERVICE_CODE AS TRAIN_SERVICE_CODE,
        TM.TOC_ID AS TOC_ID,
        TM.TOC AS TOC,
        TM.LOC_STANOX AS LOC_STANOX,
        L.DESCRIPTION AS MVT_DESCRIPTION,
        object_construct_keep_null('long',L.LONGITUDE, 'lat',L.LATITUDE)::OBJECT(long float, lat float) AS MVT_LAT_LON,
        TM.AUTO_EXPECTED AS AUTO_EXPECTED,
        TM.DIRECTION_IND AS DIRECTION_IND,
        TM.ROUTE AS ROUTE,
        TM.PLANNED_EVENT_TYPE AS PLANNED_EVENT_TYPE,
        TM.NEXT_REPORT_STANOX AS NEXT_REPORT_STANOX,
        TM.LINE_IND AS LINE_IND,
        TA.SCHEDULE_SOURCE AS SCHEDULE_SOURCE,
        TA.TP_ORIGIN_TIMESTAMP AS TP_ORIGIN_TIMESTAMP,
        TA.SCHEDULE_TYPE AS SCHEDULE_TYPE,
        TA.CREATION_TIMESTAMP AS CREATION_TIMESTAMP,
        TA.ORIGIN_DEP_TIMESTAMP AS ORIGIN_DEP_TIMESTAMP,
        TA.D1266_RECORD_NUMBER AS D1266_RECORD_NUMBER,
        TA.TRAIN_SERVICE_CODE AS TRAIN_SERVICE_CODE_02,
        TA.SCHED_ORIGIN_STANOX AS SCHED_ORIGIN_STANOX,
        TA.TRAIN_UID AS TRAIN_UID,
        TA.TRAIN_CALL_MODE AS TRAIN_CALL_MODE,
        TA.TP_ORIGIN_STANOX AS TP_ORIGIN_STANOX,
        TA.SCHEDULE_WTT_ID AS SCHEDULE_WTT_ID,
        TA.TRAIN_CALL_TYPE AS TRAIN_CALL_TYPE,
        TA.SCHEDULE_END_DATE AS SCHEDULE_END_DATE,
        COALESCE(TA.SCHEDULE_KEY,'no_schedule_activation_found') AS SCHEDULE_KEY,
        TA.SCHED_ORIGIN_DESC AS SCHED_ORIGIN_DESC,
        SCH.CIF_TRAIN_UID AS CIF_TRAIN_UID,
        SCH.NUM_STOPS AS SCHEDULE_NUM_STOPS,
        SCH.SCHEDULE_START_DATE AS SCHEDULE_START_DATE,
        SCH.CIF_STP_INDICATOR AS CIF_STP_INDICATOR,
        SCH.ATOC_CODE AS ATOC_CODE,
        SCH.TRAIN_STATUS AS TRAIN_STATUS,
        SCH.POWER_TYPE AS POWER_TYPE,
        SCH.SEATING_CLASSES AS SEATING_CLASSES,
        SCH.RESERVATIONS AS RESERVATIONS,
        SCH.SLEEPING_ACCOMODATION AS SLEEPING_ACCOMODATION,
        SCH.TRAIN_CATEGORY AS TRAIN_CATEGORY,
        SCH.ORIGIN_TIPLOC_CODE AS ORIGIN_TIPLOC_CODE,
          SCH.ORIGIN_DESCRIPTION                                    AS ORIGIN_DESCRIPTION,
          SCH.ORIGIN_LAT_LON                                    AS ORIGIN_LAT_LON,
        SCH.ORIGIN_PUBLIC_DEPARTURE_TIME AS ORIGIN_PUBLIC_DEPARTURE_TIME,
        SCH.ORIGIN_PLATFORM AS ORIGIN_PLATFORM,
        SCH.DESTINATION_TIPLOC_CODE AS DESTINATION_TIPLOC_CODE,
          SCH.DESTINATION_DESCRIPTION                               AS DESTINATION_DESCRIPTION,
          SCH.DESTINATION_LAT_LON                               AS DESTINATION_LAT_LON,
        SCH.DESTINATION_PUBLIC_ARRIVAL_TIME AS DESTINATION_PUBLIC_ARRIVAL_TIME,
        SCH.DESTINATION_PLATFORM AS DESTINATION_PLATFORM  
  FROM SILVER.TRAIN_MOVEMENTS_00 TM
      LEFT JOIN TRAIN_ACTIVATIONS TA
        ON TM.TRAIN_ID = TA.TRAIN_ID
      LEFT JOIN BRONZE.LOCATIONS_RAW L
         ON TM.loc_stanox = L.STANOX
      LEFT JOIN 
        SCHEDULE SCH 
        ON COALESCE(TA.SCHEDULE_KEY,'no_schedule_activation_found') = SCH.SCHEDULE_KEY;

# Creating the TRAIN_CANCELLATIONS Table

This cell creates a dynamic Iceberg table named `TRAIN_CANCELLATIONS` by joining cancellation events from the SILVER layer with activation, schedule, and location reference data. The resulting table provides enriched cancellation event details for analytics and reporting.

In [ ]:
CREATE OR REPLACE ICEBERG TABLE TRAIN_CANCELLATIONS(
    "MSG_HEADER" STRING(16777216),
    "TRAIN_FILE_ADDRESS" STRING(16777216),
    "TRAIN_SERVICE_CODE" STRING(16777216),
    "ORIG_LOC_STANOX" STRING(16777216),
    "TOC_ID" STRING(16777216),
    "TOC" STRING(16777216),
    "DEP_TIMESTAMP" BIGINT,
    "DIVISION_CODE" BIGINT,
    "LOC_STANOX" STRING(16777216),
    "CANX_TIMESTAMP" BIGINT,
    "CANX_REASON_CODE" STRING(16777216),
    "CANX_REASON" STRING(16777216),
    "CANCELLATION_LOCATION" STRING(16777216),
    "CANCELLATION_LAT_LON" OBJECT(long FLOAT, lat FLOAT),
    "TRAIN_ID" STRING(16777216),
    "ORIG_LOC_TIMESTAMP" TIMESTAMP_NTZ,
    "CANX_TYPE" STRING(16777216),
    "SCHEDULE_SOURCE" STRING(16777216),
    "TP_ORIGIN_TIMESTAMP" TIMESTAMP_NTZ,
    "SCHEDULE_TYPE" STRING(16777216),
    "CREATION_TIMESTAMP" TIMESTAMP_NTZ,
    "ORIGIN_DEP_TIMESTAMP" TIMESTAMP_NTZ,
    "D1266_RECORD_NUMBER" STRING(16777216),
    "TRAIN_SERVICE_CODE_02" STRING(16777216),
    "SCHED_ORIGIN_STANOX" STRING(16777216),
    "TRAIN_UID" STRING(16777216),
    "TRAIN_CALL_MODE" STRING(16777216),
    "TP_ORIGIN_STANOX" STRING(16777216),
    "SCHEDULE_WTT_ID" STRING(16777216),
    "TRAIN_CALL_TYPE" STRING(16777216),
    "SCHEDULE_END_DATE" DATE,
    "SCHEDULE_KEY" STRING(16777216) NOT NULL,
    "SCHED_ORIGIN_DESC" STRING(16777216),
    "CIF_TRAIN_UID" STRING(16777216),
    "SCHEDULE_START_DATE" DATE,
    "CIF_STP_INDICATOR" STRING(16777216),
    "ATOC_CODE" STRING(16777216),
    "TRAIN_STATUS" STRING(16777216),
    "POWER_TYPE" STRING(16777216),
    "SEATING_CLASSES" STRING(16777216),
    "RESERVATIONS" STRING(16777216),
    "SLEEPING_ACCOMODATION" STRING(16777216),
    "TRAIN_CATEGORY" STRING(16777216),
    "ORIGIN_TIPLOC_CODE" STRING(16777216),
    "ORIGIN_DESCRIPTION" STRING(16777216),
    "ORIGIN_LAT_LON" DOUBLE,
    "ORIGIN_PUBLIC_DEPARTURE_TIME" STRING(16777216),
    "ORIGIN_PLATFORM" STRING(16777216),
    "DESTINATION_TIPLOC_CODE" STRING(16777216),
    "DESTINATION_DESCRIPTION" STRING(16777216),
    "DESTINATION_LAT_LON" DOUBLE,
    "DESTINATION_PUBLIC_ARRIVAL_TIME" STRING(16777216),
    "DESTINATION_PLATFORM" STRING(16777216)
) 
EXTERNAL_VOLUME = 'iceberg_hol_gold_vol' 
CATALOG = 'SNOWFLAKE' 
BASE_LOCATION = 'hol_user_{{usernum}}/train_cancellations/' 
CATALOG_SYNC = 'iceberg_hol_oc_int' ;

In [ ]:
# Snowpark imports
from snowflake.snowpark.functions import col, coalesce, object_construct_keep_null, to_object, lit, sql_expr
from snowflake.snowpark.types import FloatType, StructField, StructType, MapType

# Load source DataFrames
tc = session.table("SILVER.TRAIN_CANCELLATIONS_00")
l = session.table("BRONZE.LOCATIONS_RAW")
c = session.table("SILVER.CANCEL_CODE_REFERENCES_00")
ta = session.table("TRAIN_ACTIVATIONS")
sch = session.table("SCHEDULE")


df_tc_l = (
            tc.join(l,col("LOC_STANOX") == col("STANOX"), how="left")
                .join(c, col("CANX_REASON_CODE") == col("CODE"), how="left")
                .join(sch,  coalesce(col("SCHEDULE_KEY"), lit("X")) == col("SCHEDULE_KEY"), how="left",rsuffix = '_sch')
                .join(ta,  col("TRAIN_ID_TA") == col("TRAIN_ID"), how="left",rsuffix = '_TA')
                .select(
                col("MSG_HEADER").alias("MSG_HEADER"),
                col("TRAIN_FILE_ADDRESS").alias("TRAIN_FILE_ADDRESS"),
                col("TRAIN_SERVICE_CODE").alias("TRAIN_SERVICE_CODE"),
                col("ORIG_LOC_STANOX").alias("ORIG_LOC_STANOX"),
                col("TOC_ID").alias("TOC_ID"),
                col("TOC").alias("TOC"),
                col("DEP_TIMESTAMP").alias("DEP_TIMESTAMP"),
                col("DIVISION_CODE").alias("DIVISION_CODE"),
                col("LOC_STANOX").alias("LOC_STANOX"),
                col("CANX_TIMESTAMP").alias("CANX_TIMESTAMP"),
                col("CANX_REASON_CODE").alias("CANX_REASON_CODE"),
                col("CODE").alias("CANX_REASON"),
                col("DESCRIPTION").alias("CANCELLATION_LOCATION"),
                sql_expr("object_construct_keep_null('long',LONGITUDE,'lat',LATITUDE)::OBJECT(long FLOAT, lat FLOAT)").alias("CANCELLATION_LAT_LON"),    
                #object_construct_keep_null(lit("long"), col("LONGITUDE"), lit("lat"), col("LATITUDE")).cast(MapType([StructField("long", FloatType()), StructField("lat", FloatType())])).alias("CANCELLATION_LAT_LON"),
                col("TRAIN_ID").alias("TRAIN_ID"),
                col("ORIG_LOC_TIMESTAMP").alias("ORIG_LOC_TIMESTAMP"),
                col("CANX_TYPE").alias("CANX_TYPE"),
                col("SCHEDULE_SOURCE").alias("SCHEDULE_SOURCE"),
                col("TP_ORIGIN_TIMESTAMP").alias("TP_ORIGIN_TIMESTAMP"),
                col("SCHEDULE_TYPE").alias("SCHEDULE_TYPE"),
                col("CREATION_TIMESTAMP").alias("CREATION_TIMESTAMP"),
                col("ORIGIN_DEP_TIMESTAMP").alias("ORIGIN_DEP_TIMESTAMP"),
                col("D1266_RECORD_NUMBER").alias("D1266_RECORD_NUMBER"),
                col("TRAIN_SERVICE_CODE").alias("TRAIN_SERVICE_CODE_02"),
                col("SCHED_ORIGIN_STANOX").alias("SCHED_ORIGIN_STANOX"),
                col("TRAIN_UID").alias("TRAIN_UID"),
                col("TRAIN_CALL_MODE").alias("TRAIN_CALL_MODE"),
                col("TP_ORIGIN_STANOX").alias("TP_ORIGIN_STANOX"),
                col("SCHEDULE_WTT_ID").alias("SCHEDULE_WTT_ID"),
                col("TRAIN_CALL_TYPE").alias("TRAIN_CALL_TYPE"),
                col("SCHEDULE_END_DATE").alias("SCHEDULE_END_DATE"),
                coalesce(col("SCHEDULE_KEY"), lit("no_schedule_found")).alias("SCHEDULE_KEY"),
                col("SCHED_ORIGIN_DESC").alias("SCHED_ORIGIN_DESC"),
                col("CIF_TRAIN_UID").alias("CIF_TRAIN_UID"),
                col("SCHEDULE_START_DATE").alias("SCHEDULE_START_DATE"),
                col("CIF_STP_INDICATOR").alias("CIF_STP_INDICATOR"),
                col("ATOC_CODE").alias("ATOC_CODE"),
                col("TRAIN_STATUS").alias("TRAIN_STATUS"),
                col("POWER_TYPE").alias("POWER_TYPE"),
                col("SEATING_CLASSES").alias("SEATING_CLASSES"),
                col("RESERVATIONS").alias("RESERVATIONS"),
                col("SLEEPING_ACCOMODATION").alias("SLEEPING_ACCOMODATION"),
                col("TRAIN_CATEGORY").alias("TRAIN_CATEGORY"),
                col("ORIGIN_TIPLOC_CODE").alias("ORIGIN_TIPLOC_CODE"),
                col("ORIGIN_DESCRIPTION").alias("ORIGIN_DESCRIPTION"),
                col("ORIGIN_LAT_LON").alias("ORIGIN_LAT_LON"),
                col("ORIGIN_PUBLIC_DEPARTURE_TIME").alias("ORIGIN_PUBLIC_DEPARTURE_TIME"),
                col("ORIGIN_PLATFORM").alias("ORIGIN_PLATFORM"),
                col("DESTINATION_TIPLOC_CODE").alias("DESTINATION_TIPLOC_CODE"),
                col("DESTINATION_DESCRIPTION").alias("DESTINATION_DESCRIPTION"),
                col("DESTINATION_LAT_LON").alias("DESTINATION_LAT_LON"),
                col("DESTINATION_PUBLIC_ARRIVAL_TIME").alias("DESTINATION_PUBLIC_ARRIVAL_TIME"),
                col("DESTINATION_PLATFORM").alias("DESTINATION_PLATFORM")
                )
        )

tc_iceberg_config = {
    "external_volume": "iceberg_hol_gold_vol",
    "catalog": "SNOWFLAKE",
    "base_location": f"hol_user_{usernum}/train_cancellations/",
    "catalog_sync": "iceberg_hol_oc_int"
}

# To create a table from this DataFrame:
df_tc_l.write.save_as_table("TRAIN_CANCELLATIONS", mode="truncate", iceberg_config = tc_iceberg_config
)


# Creating the TOC_MVT_STATS View

This step creates a summary view named `TOC_MVT_STATS` that aggregates movement statistics by Train Operating Company (TOC). The view provides counts of movements, unique trains, and time ranges, supporting dashboarding and business intelligence use cases.

In [ ]:
CREATE VIEW IF NOT EXISTS TOC_MVT_STATS AS
SELECT TOC,
  sum( case when SCHEDULE_KEY = 'no_schedule_activation_found' then 1 else 0 end ) as no_activation_found,
  sum( case when SCHEDULE_KEY = 'no_schedule_activation_found' then 0 else 1 end ) as activation_found,
  COUNT(*) as MOVEMENT_CT,
  COUNT(DISTINCT(train_id)) as unique_trains,
  TO_CHAR(
  CONVERT_TIMEZONE('UTC', 'Europe/London', TO_TIMESTAMP(min(ACTUAL_TIMESTAMP)/1000)),
  'yyyy-MM-dd HH:mm:ss'
 ) first_ts,
 TO_CHAR(
  CONVERT_TIMEZONE('UTC', 'Europe/London', TO_TIMESTAMP(max(ACTUAL_TIMESTAMP)/1000)),
  'yyyy-MM-dd HH:mm:ss'
 ) last_ts
FROM TRAIN_MOVEMENTS
GROUP BY TOC;